In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint


2023-12-07 17:31:26.421214: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 17:31:26.421256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 17:31:26.422393: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 17:31:27.461782: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Set the input size based on the pre-trained model you choose
input_size = (224, 224)

# Define data directories
train_dir = './jitropha/leaf'

# Define the batch size
batch_size = 32


In [3]:
# Use data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    rotation_range=20,
    horizontal_flip=True,
    validation_split=0.2
)


In [4]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


Found 207 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [5]:
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Create the model by adding custom classification layers
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)


In [16]:
model = Model(base_model.input, x)


In [17]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [18]:
checkpoint_callback = ModelCheckpoint(
    filepath='./Training/health_model/best_model.keras',    # Path to save the model
    monitor='val_accuracy',      # Metric to monitor for saving the best model
    save_best_only=True,         # Save only the best model (overrides previous best)
    save_weights_only=False,     # Save the entire model, not just the weights
    mode='max',                  # 'max' or 'min' depending on the monitored metric
    verbose=1                    # Display information about the saving process
)


In [19]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    verbose=1,
    callbacks=[checkpoint_callback]
)


Epoch 1/30
6/6 [==============================] - ETA: 0s - loss: 0.3237 - accuracy: 0.8743
Epoch 1: val_accuracy improved from -inf to 0.87500, saving model to ./Training/health_model/best_model.keras
6/6 [==============================] - 39s 6s/step - loss: 0.3237 - accuracy: 0.8743 - val_loss: 0.3935 - val_accuracy: 0.8750
Epoch 2/30
6/6 [==============================] - ETA: 0s - loss: 0.3270 - accuracy: 0.8686
Epoch 2: val_accuracy did not improve from 0.87500
6/6 [==============================] - 30s 5s/step - loss: 0.3270 - accuracy: 0.8686 - val_loss: 0.3755 - val_accuracy: 0.8750
Epoch 3/30
6/6 [==============================] - ETA: 0s - loss: 0.3105 - accuracy: 0.8800
Epoch 3: val_accuracy improved from 0.87500 to 0.90625, saving model to ./Training/health_model/best_model.keras
6/6 [==============================] - 33s 6s/step - loss: 0.3105 - accuracy: 0.8800 - val_loss: 0.3340 - val_accuracy: 0.9062
Epoch 4/30
6/6 [==============================] - ETA: 0s - loss: 0.3

In [20]:
model.evaluate(validation_generator, verbose=1)


2/2 [==============================] - 7s 2s/step - loss: 0.3081 - accuracy: 0.9200


[0.30805930495262146, 0.9200000166893005]